In [14]:
import fitz  # PyMuPDF
import pandas as pd
import re
# Open the PDF file
PDF_PATH = 'Test PDF.pdf'
DB_PATH = 'SQLITE_DB.db'



def get_data_from_pdf(pdf_path=PDF_PATH):
    pdf_document = fitz.open(pdf_path)

    # Extract text from the first page as an example
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()

    pdf_document.close()
    
    # Define a pattern to identify and split the data entries
    # This pattern assumes specific formatting and may need adjustments if there are variations in the text structure
    entry_pattern = re.compile(r'\n(\d{8})\s*(\d{9})\s*(\d{1,2}/\d{1,2}/\d{4})\s*([A-Z].*)\s*( ?|[A-Z-]?[a-z-]*(?: +[A-Z-][a-z-]*[A-Z]?[a-z]*)*)\s*([A-Z-]+(?:\s+[A-Z-]+)*)\s*([A-Z].*)\s*([\d,]+.\d{2})\s*(\d.\d{2})\s*([\d,]+.\d{2})\s*([\d,]+.\d{2})')

    # Find all matches of the pattern in the text
    matches = entry_pattern.findall(text)

    # Convert the matches into a DataFrame
    df_columns = ['App ID', 'Xref', 'Date', 'Broker', 'Sub Broker', 'Borrower Name', 'Description', 'Total Loan Amount', 'Comm Rate', 'Upfront', 'Upfront Incl GST']
    df = pd.DataFrame(matches, columns=df_columns)

    # Convert numerical columns to the appropriate data types and handle comma in numbers
    for col in ['Total Loan Amount', 'Comm Rate', 'Upfront', 'Upfront Incl GST']:
        df[col] = df[col].str.replace(',', '').astype(float)

    return df
    
    
    


In [15]:
import sqlite3

def insert_data_into_sqlite(df, db_path=DB_PATH):
    """
    Inserts data into a SQLite database with deduplication based on Xref and Total Loan Amount.
    If the table does not exist, it will be created.

    Parameters:
    - df: The DataFrame containing the data to insert.
    - db_path: Path to the SQLite database file.
    """
    # Connect to SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Check if the table exists, if not, create it
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='transactions'")
    if cursor.fetchone() is None:
        create_table_query = '''
        CREATE TABLE transactions (
            app_id TEXT,
            xref TEXT,
            date TEXT,
            broker TEXT,
            sub_broker TEXT,
            borrower_name TEXT,
            description TEXT,
            total_loan_amount REAL,
            comm_rate REAL,
            upfront REAL,
            upfront_incl_gst REAL,
            UNIQUE(xref, total_loan_amount) ON CONFLICT IGNORE
        )
        '''
        cursor.execute(create_table_query)

    # Insert data into the table, ignoring duplicates based on Xref and Total Loan Amount
    insert_query = '''
    INSERT OR IGNORE INTO transactions (app_id, xref, date, broker, sub_broker, borrower_name, description, total_loan_amount, comm_rate, upfront, upfront_incl_gst)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    cursor.executemany(insert_query, df.values.tolist())

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

In [16]:
def calculate_total_loan_amount(start_date, end_date, db_path=DB_PATH):
    """
    Calculates the total loan amount for a specific time period.

    Parameters:
    - db_path: Path to the SQLite database file.
    - start_date: Start date of the period (inclusive) in 'YYYY-MM-DD' format.
    - end_date: End date of the period (inclusive) in 'YYYY-MM-DD' format.

    Returns:
    - Total loan amount for the specified period.
    """
    conn = sqlite3.connect(db_path)
    query = '''
    SELECT SUM(total_loan_amount) FROM transactions
    WHERE date BETWEEN ? AND ?
    '''
    total_loan_amount = conn.execute(query, (start_date, end_date)).fetchone()[0]
    conn.close()
    return total_loan_amount

def calculate_highest_loan_by_broker(db_path=DB_PATH):
    """
    Calculates the highest loan amount given by a broker.

    Parameters:
    - db_path: Path to the SQLite database file.

    Returns:
    - Broker name and the highest loan amount given by that broker.
    """
    conn = sqlite3.connect(db_path)
    query = '''
    SELECT broker, MAX(total_loan_amount) FROM transactions
    GROUP BY broker
    ORDER BY MAX(total_loan_amount) DESC
    LIMIT 1
    '''
    result = conn.execute(query).fetchone()
    conn.close()
    return result




In [17]:
def generate_reports(db_path=DB_PATH):
    """
    Generates various reports from the transactions database.
    
    Parameters:
    - db_path: Path to the SQLite database file.
    
    Returns:
    - A dictionary containing all the requested reports.
    """
    conn = sqlite3.connect(db_path)
    reports = {}

    # Report 1: Broker loan amounts in descending order for daily periods
    reports['daily_broker_loan_amounts'] = pd.read_sql_query(
        '''
        SELECT date, broker, SUM(total_loan_amount) as total_loan_amount
        FROM transactions
        GROUP BY date, broker
        ORDER BY date, total_loan_amount DESC;
        ''', conn)

    # Report 2: Total loan amount grouped by date
    reports['total_loan_amount_by_date'] = pd.read_sql_query(
        '''
        SELECT date, SUM(total_loan_amount) as total_loan_amount
        FROM transactions
        GROUP BY date
        ORDER BY date;
        ''', conn)

    # Report 3: Tier level of each transaction
    reports['transaction_tiers'] = pd.read_sql_query(
        '''
        SELECT *, 
        CASE
            WHEN total_loan_amount > 100000 THEN 'Tier 1'
            WHEN total_loan_amount > 50000 THEN 'Tier 2'
            ELSE 'Tier 3'
        END as tier
        FROM transactions;
        ''', conn)

    # Report 4: Number of loans under each tier group by date
    reports['loans_by_tier_and_date'] = pd.read_sql_query(
        '''
        SELECT date,
        SUM(CASE WHEN total_loan_amount > 100000 THEN 1 ELSE 0 END) AS Tier_1,
        SUM(CASE WHEN total_loan_amount > 50000 THEN 1 ELSE 0 END) AS Tier_2,
        SUM(CASE WHEN total_loan_amount > 10000 THEN 1 ELSE 0 END) AS Tier_3
        FROM transactions
        GROUP BY date
        ORDER BY date;
        ''', conn)

    conn.close()
    return reports



In [18]:
data = get_data_from_pdf()
insert_data_into_sqlite(data)
START_DATE = '17/10/2023'
END_DATE = '25/10/2023'
total_loan_amount = calculate_total_loan_amount(START_DATE, END_DATE)
print(f"Total loan amount from {START_DATE} to {END_DATE}: {total_loan_amount}")
highest_loan_by_broker = calculate_highest_loan_by_broker()
print(f"Broker with the highest loan amount: {highest_loan_by_broker[0]}, Loan Amount: {highest_loan_by_broker[1]}")


Total loan amount from 17/10/2023 to 25/10/2023: 376868.93
Broker with the highest loan amount: Stratton Norwest , Loan Amount: 77590.87


In [19]:
reports = generate_reports()
reports.keys()

dict_keys(['daily_broker_loan_amounts', 'total_loan_amount_by_date', 'transaction_tiers', 'loans_by_tier_and_date'])

In [20]:
reports['daily_broker_loan_amounts']

,date,broker,total_loan_amount
0,10/10/2023,F1 Finance Pty Ltd,53600.00
1,10/10/2023,Stratton Norwest,43890.00
2,10/10/2023,Statewide Lending Pty Ltd,15530.00
3,11/10/2023,Stratton Norwest,85756.25
4,11/10/2023,Stratton Finance Buderim,53910.00
5,12/10/2023,Stratton Norwest,77590.87
6,13/10/2023,Stratton Finance,66585.00
7,16/10/2023,Auswide Financial Solutions Pty Ltd,38770.00
8,17/10/2023,Auswide Financial Solutions Pty Ltd,17168.11
9,18/10/2023,Auswide Financial Solutions Pty Ltd,54342.99


In [21]:
reports['total_loan_amount_by_date']

,date,total_loan_amount
0,10/10/2023,113020.00
1,11/10/2023,139666.25
2,12/10/2023,77590.87
3,13/10/2023,66585.00
4,16/10/2023,38770.00
5,17/10/2023,17168.11
6,18/10/2023,54342.99
7,24/10/2023,102210.41
8,25/10/2023,203147.42
9,27/10/2023,194143.16


In [22]:
reports['transaction_tiers']

,app_id,xref,date,broker,sub_broker,borrower_name,description,total_loan_amount,comm_rate,upfront,upfront_incl_gst,tier
0,80187668,100305742,18/10/2023,Auswide Financial Solutions Pty Ltd,Carole Leedham,ALANA MANZOTTI,Upfront Commission,54342.99,1.8,978.17,1075.99,Tier 2
1,80189176,100306768,25/10/2023,Stratton Clark,Faith Johnson,GRAEME SMITH,Upfront Commission,45188.00,1.8,813.38,894.72,Tier 3
2,80183969,100306746,25/10/2023,Stratton Finance Pty Ltd,Benjamin Ly,PETER SAMAAN,Upfront Commission,39100.00,1.8,703.80,774.18,Tier 3
3,80188829,100306766,25/10/2023,Stratton Norwest,Mitchell Mulach,ARON BARJAKTAREVIC,Upfront Commission,51774.35,1.8,931.94,1025.13,Tier 2
4,80187448,100306612,24/10/2023,Stratton Albury Wodonga,Melissa Dolden,ALEX HARRY VANGALEN,Upfront Commission,32975.00,1.8,593.55,652.91,Tier 3
5,80188025,100306545,24/10/2023,Stratton Norwest,Mitchell Mulach,BRYONIE GEORGIA MARY WHITE,Upfront Commission,51390.00,1.8,925.02,1017.52,Tier 2
6,80187510,100306559,24/10/2023,Matocanza Family Trust,Sarah Lynch,BRADLEY IAN NELSON,Upfront Commission,17845.41,1.8,321.22,353.34,Tier 3
7,80189985,100307015,27/10/2023,Auswide Financial Solutions Pty Ltd,Carole Leedham,THEO COUVARDIS,Upfront Commission,35000.00,1.8,630.00,693.00,Tier 3
8,80188006,100307016,27/10/2023,Stratton Norwest,Mitchell Mulach,JACK RICARDO,Upfront Commission,47022.46,1.8,846.40,931.04,Tier 3
9,80186584,100307113,27/10/2023,Stratton Norwest,Mitchell Mulach,ANTHONY MCKEOWN,Upfront Commission,67535.70,1.8,1215.64,1337.20,Tier 2


In [23]:
reports['loans_by_tier_and_date']

,date,Tier_1,Tier_2,Tier_3
0,10/10/2023,0,1,3
1,11/10/2023,0,2,3
2,12/10/2023,0,1,1
3,13/10/2023,0,1,1
4,16/10/2023,0,0,1
5,17/10/2023,0,0,1
6,18/10/2023,0,1,1
7,24/10/2023,0,1,3
8,25/10/2023,0,2,4
9,27/10/2023,0,1,4
